# Backtesting an investment strategy based on model predictions
In this notebook we will compare the performance of a buy/sell strategy according to the predictions of opur model versus a simple buy and hold strategy.

In [61]:
import pandas as pd
import numpy as np
import ffn
import bt

pd.set_option('display.max_rows', 15)

## Bitcoin prices

In [19]:
prices = pd.read_csv("Data/poloniex_data.csv")

In [24]:
prices = prices[["close","date"]]

In [25]:
prices

,close,date
301,10338.048837,2018-02-26
302,10589.174340,2018-02-27
303,10322.067296,2018-02-28
304,10936.784498,2018-03-01
305,11022.243806,2018-03-02
306,11441.000001,2018-03-03
307,11500.605200,2018-03-04
...,...,...
357,8934.977783,2018-04-23
358,9633.630952,2018-04-24


## Predictions

In [14]:
predictions = pd.read_csv("Data/nonlagged_predictions.csv", index_col = 0)

In [16]:
predictions

,predict,False,True
0,False,0.543595,0.456405
1,True,0.395375,0.604625
2,True,0.434563,0.565437
3,True,0.380255,0.619745
4,True,0.427909,0.572091
5,True,0.399354,0.600646
6,True,0.335182,0.664818
...,...,...,...
56,True,0.277206,0.722794
57,True,0.431252,0.568748


In [50]:
import pandas as pd
datelist = pd.date_range("26-02-2018", periods=63).tolist()

In [51]:
predictions["date"] = datelist

In [53]:
predictions = predictions.set_index("date")

In [54]:
predictions

,predict,False,True
date,,,
2018-02-26,False,0.543595,0.456405
2018-02-27,True,0.395375,0.604625
2018-02-28,True,0.434563,0.565437
2018-03-01,True,0.380255,0.619745
2018-03-02,True,0.427909,0.572091
2018-03-03,True,0.399354,0.600646
2018-03-04,True,0.335182,0.664818
...,...,...,...
2018-04-23,True,0.277206,0.722794


## Adapting prices time range to fit with test set.

The predictions we have only represent the last 63 days. But our data stopped at the 29/04/2018. ** We probably lost some data when doing dropna() **

In [27]:
prices = prices[:364]

In [28]:
prices

,close,date
301,10338.048837,2018-02-26
302,10589.174340,2018-02-27
303,10322.067296,2018-02-28
304,10936.784498,2018-03-01
305,11022.243806,2018-03-02
306,11441.000001,2018-03-03
307,11500.605200,2018-03-04
...,...,...
357,8934.977783,2018-04-23
358,9633.630952,2018-04-24


In [29]:
prices = prices[-63:]

In [39]:
prices

,close,date
301,10338.048837,2018-02-26
302,10589.174340,2018-02-27
303,10322.067296,2018-02-28
304,10936.784498,2018-03-01
305,11022.243806,2018-03-02
306,11441.000001,2018-03-03
307,11500.605200,2018-03-04
...,...,...
357,8934.977783,2018-04-23
358,9633.630952,2018-04-24


In [55]:
prices = prices.set_index("date")
prices

,close
date,
2018-02-26,10338.048837
2018-02-27,10589.174340
2018-02-28,10322.067296
2018-03-01,10936.784498
2018-03-02,11022.243806
2018-03-03,11441.000001
2018-03-04,11500.605200
...,...
2018-04-23,8934.977783


## Merge tables

In [79]:
data = pd.concat((prices, predictions), axis = 1)
data.to_csv("Data/backtest_data.csv")
data

,close,predict,False,True
date,,,,
2018-02-26,10338.048837,False,0.543595,0.456405
2018-02-27,10589.174340,True,0.395375,0.604625
2018-02-28,10322.067296,True,0.434563,0.565437
2018-03-01,10936.784498,True,0.380255,0.619745
2018-03-02,11022.243806,True,0.427909,0.572091
2018-03-03,11441.000001,True,0.399354,0.600646
2018-03-04,11500.605200,True,0.335182,0.664818
...,...,...,...,...
2018-04-23,8934.977783,True,0.277206,0.722794


## Backtesting the investment strategy

In [80]:
tw = data[["predict"]]  #"False",  True","False
tw

,predict
date,
2018-02-26,False
2018-02-27,True
2018-02-28,True
2018-03-01,True
2018-03-02,True
2018-03-03,True
2018-03-04,True
...,...
2018-04-23,True


In [81]:
btc_price = data[["close"]]
btc_price

,close
date,
2018-02-26,10338.048837
2018-02-27,10589.174340
2018-02-28,10322.067296
2018-03-01,10936.784498
2018-03-02,11022.243806
2018-03-03,11441.000001
2018-03-04,11500.605200
...,...
2018-04-23,8934.977783


In [82]:
# create the strategy
strategy = bt.Strategy('Model Advice Strategy', [bt.algos.WeighTarget(tw),
                                                 bt.algos.Rebalance()])

t = bt.Backtest(strategy, btc_price)
res = bt.run(t)

TypeError: unsupported operand type(s) for -: 'str' and 'relativedelta'